In [14]:
from collections import OrderedDict

import h5py
import torch

spec_to_i = {
    'tiny': '_3',
    'small': '_4',
    'medium': '_5',
    'large': '_6',
    'full': ''
}

for spec in ['full']:#['tiny', 'small', 'medium', 'large', 'full']:
    h5_fn = 'model-{}.h5'.format(spec)
    py_fn = '/home/cdonahue/coltrain/coltrain/params/crepe_{}.bin'.format(spec)
    i = spec_to_i[spec]

    d = OrderedDict()

    H5_NAME_TO_PYTORCH_NAME = {
        ('classifier', 'classifier{}'.format(i), 'kernel'): 'classifier.weight',
        ('classifier', 'classifier{}'.format(i), 'bias'): 'classifier.bias',
    }
    
    for l in [1, 2, 3, 4, 5, 6]:
        H5_NAME_TO_PYTORCH_NAME.update({
            ('conv{}'.format(l), 'conv{}{}'.format(l, i), 'kernel'): 'conv{}.weight'.format(l),
            ('conv{}'.format(l), 'conv{}{}'.format(l, i), 'bias'): 'conv{}.bias'.format(l),
            ('conv{}-BN'.format(l), 'conv{}-BN{}'.format(l, i), 'gamma'): 'conv{}-BN.weight'.format(l),
            ('conv{}-BN'.format(l), 'conv{}-BN{}'.format(l, i), 'beta'): 'conv{}-BN.bias'.format(l),
            ('conv{}-BN'.format(l), 'conv{}-BN{}'.format(l, i), 'moving_mean'): 'conv{}-BN.running_mean'.format(l),
            ('conv{}-BN'.format(l), 'conv{}-BN{}'.format(l, i), 'moving_variance'): 'conv{}-BN.running_var'.format(l)
        })

    with h5py.File(h5_fn, 'r') as f:
        for hk, pk in H5_NAME_TO_PYTORCH_NAME.items():
            arr = f[hk[0]][hk[1]][hk[2] + ':0'][()]
            if arr.ndim == 2:
                arr = arr.transpose(1, 0)
            if arr.ndim == 4:
                arr = arr.transpose(3, 2, 0, 1)
            print('-' * 80)
            print('{}->{}'.format(hk, pk))
            print(arr.shape)
            d[pk] = torch.tensor(arr, dtype=torch.float32, requires_grad=False, device='cpu')

    torch.save(d, py_fn)

--------------------------------------------------------------------------------
('classifier', 'classifier', 'kernel')->classifier.weight
(360, 2048)
--------------------------------------------------------------------------------
('classifier', 'classifier', 'bias')->classifier.bias
(360,)
--------------------------------------------------------------------------------
('conv1', 'conv1', 'kernel')->conv1.weight
(1024, 1, 512, 1)
--------------------------------------------------------------------------------
('conv1', 'conv1', 'bias')->conv1.bias
(1024,)
--------------------------------------------------------------------------------
('conv1-BN', 'conv1-BN', 'gamma')->conv1-BN.weight
(1024,)
--------------------------------------------------------------------------------
('conv1-BN', 'conv1-BN', 'beta')->conv1-BN.bias
(1024,)
--------------------------------------------------------------------------------
('conv1-BN', 'conv1-BN', 'moving_mean')->conv1-BN.running_mean
(1024,)
--------